In [ ]:
!pip install selenium

In [ ]:
import bs4
import selenium

import json
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import requests, csv, re

import pandas as pd
import numpy as np
import time
import random
from urllib.parse import urlparse, parse_qs

# 모든 기사 크롤링

In [ ]:
# total_article_list = []
# prev_article_list = []
# for date in pd.date_range('20221005','20221006').format(formatter=lambda x: x.strftime('%Y%m%d')):   ##########시작일, 종료일 입력!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#     article_page_num = 1
#     category_url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=269&sid1=100&date={}&page={}'

#     ################카테고리 바꾸기!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
#     # 정치: sid2 = 269  sid1 = 100
#     # 경제: 263   101
#     # 사회: 257   102
#     # 생활/문화: 245    103
#     # IT/과학: 322  104
#     # 세계: 228   105

#     header = {
#         'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
#     }


#     while(True):
#         req_url = category_url.format(date, article_page_num)
#         html = requests.get(req_url, headers = header)
#         soup = BeautifulSoup(html.content, 'html.parser')
#         temp_article_list = list(set([url_info.find('a').get('href') for url_info in soup.find_all('dt')]))


#         if(temp_article_list == prev_article_list):
#             break
#         else:
#             prev_article_list = list(temp_article_list)
#             total_article_list.extend(temp_article_list)
#             article_page_num+=1
#         print(len(total_article_list))

# 하루 기사 수 1400개로 제한

In [ ]:
total_article_list = []
prev_article_list = []
for date in pd.date_range('20221005','20221006').format(formatter=lambda x: x.strftime('%Y%m%d')):   ##########시작일, 종료일 입력!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    test_article_list = []
    article_page_num = 1
    category_url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=269&sid1=100&date={}&page={}'

    ################카테고리 바꾸기!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
    # 정치: sid1 = 269  sid2 = 100
    # 경제: 263   101
    # 사회: 257   102
    # 생활/문화: 245    103
    # 세계: 322  104
    # IT/과학: 228   105

    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
    }


    while(True):
        req_url = category_url.format(date, article_page_num)
        html = requests.get(req_url, headers = header)
        soup = BeautifulSoup(html.content, 'html.parser')
        temp_article_list = list(set([url_info.find('a').get('href') for url_info in soup.find_all('dt')]))


        if(temp_article_list == prev_article_list):
            break
        else:
            prev_article_list = list(temp_article_list)
            total_article_list.extend(temp_article_list)
            test_article_list.extend(temp_article_list)
            article_page_num+=1
        print(len(total_article_list))
        if len(test_article_list)>1399:
            break

In [ ]:
comment_all_list = []
for article_url in tqdm(total_article_list):

    oid = article_url[39:42]
    aid = article_url[43:53]


    page_num = 1
    url ="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(page_num) + "&refresh=false&sort=FAVORITE"
    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
        'accept' : "*/*",
        'accept-encoding' : 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'referer' : article_url
    }

    comment_list = []
    while(True):

        html = requests.get(url, headers = header)

        test_text = html.text
        test_text = test_text.replace("jQuery1707138182064460843_1523512042464(","")[:-2]
        comment_text = json.loads(test_text)


        temp_comment_list = [comment_info['contents'] for comment_info in comment_text['result']['commentList']]
        comment_list.extend(temp_comment_list)

        if(len(temp_comment_list) < 100):
            break
        page_num+=1
    comment_all_list.extend(comment_list)
    time.sleep(random.uniform(0.5, 1))

In [ ]:
comment_df = pd.DataFrame(comment_all_list , columns = ['comment'])
comment_df

In [ ]:
import pickle

comment_df.to_pickle("df.pkl")  ###########파일이름!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # 정치: 1~
    # 경제: 100~
    # 사회: 200~
    # 생활/문화: 300~
    # 세계: 400~
    # IT/과학: 500~]

    #Ex)정치파일:  comment_df.to_pickle("13.pkl")


# 크롤링 잘 돼고 있는지 가끔 파일 열어서 확인해주기!!!!!!!!!!!!!!!!!!!!!!!!!!1
#df = pd.read_pickle("#저장한 파일이름")
#df